In [14]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
import impyute

In [28]:
from ingestion import *

In [15]:
csv_name = "data_raw/dhw_5km_db8d_5daf_80ed.csv"

In [16]:
df = pd.read_csv(csv_name, index_col=False)
df.head()

,time,latitude,longitude,CRW_BAA,CRW_BAA_mask,CRW_BAA_7D_MAX,CRW_BAA_7D_MAX_mask,CRW_DHW,CRW_DHW_mask,CRW_HOTSPOT,CRW_HOTSPOT_mask,CRW_SST,CRW_SSTANOMALY,CRW_SSTANOMALY_mask
0,2022-05-27T12:00:00Z,18.375,-65.625,0,0,0,0,0.0,0,-0.98,0,27.57,0.34,0
1,2022-05-27T12:00:00Z,18.375,-65.575,0,0,0,0,0.0,0,-0.96,0,27.58,0.34,0
2,2022-05-27T12:00:00Z,18.375,-65.525,0,0,0,0,0.0,0,-0.95,0,27.58,0.33,0
3,2022-05-27T12:00:00Z,18.325,-65.625,0,0,0,0,0.0,0,-0.88,0,27.68,0.41,0
4,2022-05-27T12:00:00Z,18.325,-65.575,0,0,0,0,0.0,0,-0.86,0,27.69,0.41,0


In [17]:
df.columns = df.columns.str.strip().str.lower()

In [18]:
df['time'] = pd.to_datetime(df['time']).dt.date
df.head()

,time,latitude,longitude,crw_baa,crw_baa_mask,crw_baa_7d_max,crw_baa_7d_max_mask,crw_dhw,crw_dhw_mask,crw_hotspot,crw_hotspot_mask,crw_sst,crw_sstanomaly,crw_sstanomaly_mask
0,2022-05-27,18.375,-65.625,0,0,0,0,0.0,0,-0.98,0,27.57,0.34,0
1,2022-05-27,18.375,-65.575,0,0,0,0,0.0,0,-0.96,0,27.58,0.34,0
2,2022-05-27,18.375,-65.525,0,0,0,0,0.0,0,-0.95,0,27.58,0.33,0
3,2022-05-27,18.325,-65.625,0,0,0,0,0.0,0,-0.88,0,27.68,0.41,0
4,2022-05-27,18.325,-65.575,0,0,0,0,0.0,0,-0.86,0,27.69,0.41,0


In [19]:
df = df.rename(columns={'time': 'date'}, errors='raise')
df.columns

Index(['date', 'latitude', 'longitude', 'crw_baa', 'crw_baa_mask',
       'crw_baa_7d_max', 'crw_baa_7d_max_mask', 'crw_dhw', 'crw_dhw_mask',
       'crw_hotspot', 'crw_hotspot_mask', 'crw_sst', 'crw_sstanomaly',
       'crw_sstanomaly_mask'],
      dtype='object')

In [20]:
df.dtypes

date                    object
latitude               float64
longitude              float64
crw_baa                  int64
crw_baa_mask             int64
crw_baa_7d_max           int64
crw_baa_7d_max_mask      int64
crw_dhw                float64
crw_dhw_mask             int64
crw_hotspot            float64
crw_hotspot_mask         int64
crw_sst                float64
crw_sstanomaly         float64
crw_sstanomaly_mask      int64
dtype: object

In [21]:
# Add synthetic missing values
ewm_impute_cols = ['crw_hotspot', 'crw_sst', 'crw_sstanomaly']
for col in ewm_impute_cols:
        df[col].iloc[-1] = np.nan

C:\Users\Bryan\AppData\Local\Temp\ipykernel_37964\618171366.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].iloc[-1] = np.nan


In [22]:
df.iloc[-1:]

,date,latitude,longitude,crw_baa,crw_baa_mask,crw_baa_7d_max,crw_baa_7d_max_mask,crw_dhw,crw_dhw_mask,crw_hotspot,crw_hotspot_mask,crw_sst,crw_sstanomaly,crw_sstanomaly_mask
2195,2023-05-27,18.325,-65.525,0,0,0,0,0.0,0,NaN,0,NaN,NaN,0


In [23]:
# View missing values
for col in df.columns:
    print(f"{col}: {df[col].isnull().sum()}")

date: 0
latitude: 0
longitude: 0
crw_baa: 0
crw_baa_mask: 0
crw_baa_7d_max: 0
crw_baa_7d_max_mask: 0
crw_dhw: 0
crw_dhw_mask: 0
crw_hotspot: 1
crw_hotspot_mask: 0
crw_sst: 1
crw_sstanomaly: 1
crw_sstanomaly_mask: 0


In [24]:
# Impute missing values w/ exponential moving avg
for col in df.columns:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].ewm(span=10).mean())
    else:
        pass

In [25]:
# View final row values
df.iloc[-3:]

,date,latitude,longitude,crw_baa,crw_baa_mask,crw_baa_7d_max,crw_baa_7d_max_mask,crw_dhw,crw_dhw_mask,crw_hotspot,crw_hotspot_mask,crw_sst,crw_sstanomaly,crw_sstanomaly_mask
2193,2023-05-27,18.325,-65.625,0,0,0,0,0.0,0,-0.170000,0,28.390000,1.120000,0
2194,2023-05-27,18.325,-65.575,0,0,0,0,0.0,0,-0.160000,0,28.390000,1.110000,0
2195,2023-05-27,18.325,-65.525,0,0,0,0,0.0,0,-0.166062,0,28.380211,1.128554,0


In [26]:
df.dtypes

date                    object
latitude               float64
longitude              float64
crw_baa                  int64
crw_baa_mask             int64
crw_baa_7d_max           int64
crw_baa_7d_max_mask      int64
crw_dhw                float64
crw_dhw_mask             int64
crw_hotspot            float64
crw_hotspot_mask         int64
crw_sst                float64
crw_sstanomaly         float64
crw_sstanomaly_mask      int64
dtype: object

In [27]:
# Recount missing values
for col in df.columns:
    print(f"{col}: {df[col].isnull().sum()}")

date: 0
latitude: 0
longitude: 0
crw_baa: 0
crw_baa_mask: 0
crw_baa_7d_max: 0
crw_baa_7d_max_mask: 0
crw_dhw: 0
crw_dhw_mask: 0
crw_hotspot: 0
crw_hotspot_mask: 0
crw_sst: 0
crw_sstanomaly: 0
crw_sstanomaly_mask: 0
